In [ ]:
pip install streamlit

In [7]:
# ANEXO V - Dashboard Funcional Streamlit (con datos simulados)
# Para ejecutar: Guarda este código como un archivo .py (ej. dashboard_tfm.py)
# navega a la carpeta donde lo guardaste
# Ejecuta: streamlit run dashboard_tfm.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc # Para métricas de rendimiento simuladas

# --- Simulación de Datos ---
# Valores generales
k_actual = 10
l_actual = 2
epsilon_actual = 2.0
riesgo_estimado = "Bajo"
precision_anon = 78.9
precision_no_anon = 85.4

# Datos simulados para histograma de k-anonimato
np.random.seed(42) # Para reproducibilidad
# Genera 100 grupos que cumplen k=10 (tamaños entre 10 y 14)
group_sizes_ok = np.random.randint(k_actual, k_actual + 5, size=100)
# Genera 5 grupos que NO cumplen k=10 (tamaños entre 1 y 9)
group_sizes_nok = np.random.randint(1, k_actual, size=5)
group_sizes = np.concatenate((group_sizes_ok, group_sizes_nok))

# Datos simulados para tabla de l-diversidad
l_diversity_data = {
    'Grupo ID': [f'Grupo_{i+1}' for i in range(5)],
    'Atributo Sensible (type)': ['TRANSFER', 'CASH_OUT', 'TRANSFER', 'PAYMENT', 'CASH_OUT'],
    'Valores Distintos (l)': [2, 3, 2, 1, 2], # Grupo 4 no cumpliría l=2
    'Cumple L=2': ['Sí', 'Sí', 'Sí', 'No', 'Sí']
}
df_l_diversity = pd.DataFrame(l_diversity_data)

# Datos simulados para rendimiento (matriz confusión y ROC)
# Supongamos 1000 predicciones del modelo anonimizado
y_true_sim = np.random.randint(0, 2, size=1000) # Etiquetas verdaderas (0 o 1)
# Simulamos probabilidades predichas (ajustadas para que la precisión sea ~78.9%)
# Generamos predicciones aleatorias y luego las ajustamos
y_pred_proba_sim = np.random.rand(1000)
# Ajustamos para que coincida aproximadamente con la precisión
threshold = 0.5 # Umbral de clasificación
y_pred_sim = (y_pred_proba_sim > threshold).astype(int)
# Forzamos que ~78.9% coincidan (esto es una simplificación burda)
correct_indices = np.random.choice(1000, size=int(1000 * precision_anon / 100), replace=False)
incorrect_indices = np.setdiff1d(np.arange(1000), correct_indices)
y_pred_sim[correct_indices] = y_true_sim[correct_indices]
y_pred_sim[incorrect_indices] = 1 - y_true_sim[incorrect_indices] # Asegura que sean incorrectas

# Calcular matriz de confusión y curva ROC
cm = confusion_matrix(y_true_sim, y_pred_sim)
fpr, tpr, thresholds = roc_curve(y_true_sim, y_pred_proba_sim)
roc_auc = auc(fpr, tpr)

# Datos simulados para logs de auditoría
log_data = {
    'Fecha': pd.to_datetime(['2025-04-15', '2025-04-20', '2025-05-01']),
    'Acción': ['Entrenamiento Inicial', 'Ajuste Parámetro ε', 'Reentrenamiento Modelo'],
    'Usuario': ['DataScientist1', 'ComplianceOfficer', 'DataScientist1'],
    'Detalle': ['Modelo base entrenado', 'ε cambiado de 3.0 a 2.0', 'Modelo reentrenado con nuevos datos']
}
df_logs = pd.DataFrame(log_data)

# --- Diseño del Dashboard ---
st.set_page_config(layout="wide", page_title="Dashboard GDPR TFM")
st.title("Dashboard de Cumplimiento GDPR y Rendimiento del Modelo")
st.markdown("*Nota: Este dashboard utiliza datos simulados con fines ilustrativos.*")

# --- Pestañas ---
tab1, tab2, tab3, tab4, tab5 = st.tabs([
    "📊 Vista General",
    "🛡️ Detalle Anonimización",
    "🔒 Privacidad Diferencial",
    "📈 Rendimiento Modelo",
    "📜 Auditoría"
])

# --- Contenido Pestaña 1: Vista General ---
with tab1:
    st.header("Resumen Ejecutivo de Cumplimiento")
    col1, col2, col3, col4 = st.columns(4)
    col1.metric("K-Anonimato Actual", k_actual, help="Mínimo tamaño de grupo garantizado.")
    col2.metric("L-Diversidad Mín.", l_actual, help="Mínima diversidad de atributos sensibles requerida por grupo.")
    col3.metric("Épsilon (DP)", f"{epsilon_actual:.1f}", help="Presupuesto de privacidad diferencial.")
    col4.metric("Riesgo Reidentificación", riesgo_estimado, delta_color="off", help="Estimación cualitativa del riesgo residual.")

    st.divider() # Separador visual

    st.subheader("Comparación Precisión (Utilidad vs Privacidad)")
    df_prec = pd.DataFrame({
        'Modelo': ['Sin Anonimización', f'Con Anonimización y DP (ε={epsilon_actual})'],
        'Precisión (%)': [precision_no_anon, precision_anon],
        #'Nivel Protección GDPR': ['Bajo', 'Alto'] # Se puede añadir como leyenda o color
    })
    fig_prec, ax_prec = plt.subplots(figsize=(8, 4)) # Ajustar tamaño
    colors = ['#d9534f', '#5cb85c'] # Rojo, Verde
    bars = ax_prec.bar(df_prec['Modelo'], df_prec['Precisión (%)'], color=colors, width=0.6) # Barras más estrechas
    ax_prec.set_ylim(0, 100)
    ax_prec.set_ylabel("Precisión (%)")
    ax_prec.set_title("Comparación Utilidad vs. Nivel de Protección GDPR")
    # Añadir etiquetas de valor sobre las barras
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval + 1, f'{yval:.1f}%', va='bottom', ha='center', fontsize=10) # Ajustar posición y tamaño

    # Añadir leyenda simple
    handles = [plt.Rectangle((0,0),1,1, color=c) for c in colors]
    labels= ["Protección GDPR: Baja", "Protección GDPR: Alta"]
    ax_prec.legend(handles, labels, title="Nivel de Protección", loc='lower right')

    st.pyplot(fig_prec)


# --- Contenido Pestaña 2: Detalle Anonimización ---
with tab2:
    st.header("Análisis de Técnicas de Anonimización")

    st.subheader(f"Distribución Grupos K-Anonimato (k={k_actual})")
    fig_k, ax_k = plt.subplots(figsize=(8, 4))
    # Contar frecuencias para el histograma
    bins = range(1, int(max(group_sizes)) + 2)
    counts, edges, patches = ax_k.hist(group_sizes, bins=bins, align='left', rwidth=0.8, color='skyblue', edgecolor='black')
    ax_k.axvline(k_actual, color='r', linestyle='dashed', linewidth=1.5, label=f'Umbral k={k_actual}')
    ax_k.set_xlabel("Tamaño del Grupo de Equivalencia")
    ax_k.set_ylabel("Número de Grupos")
    ax_k.set_title("Histograma de Tamaños de Grupo")
    ax_k.set_xticks(bins[:-1]) # Asegurar ticks enteros
    ax_k.legend()
    st.pyplot(fig_k)
    # Contar cuántos grupos no cumplen k
    grupos_no_cumplen_k = sum(1 for size in group_sizes if size < k_actual)
    if grupos_no_cumplen_k > 0:
        st.warning(f"⚠️ ¡Atención! Se detectaron {grupos_no_cumplen_k} grupos con tamaño menor que k={k_actual}.")
    else:
        st.success(f"✅ Todos los grupos analizados cumplen con k={k_actual}.")
    st.caption("Idealmente, no debería haber barras a la izquierda de la línea roja.")

    st.divider()

    st.subheader(f"Verificación L-Diversidad (l={l_actual})")
    st.dataframe(df_l_diversity.style.applymap(lambda x: 'color: red' if x=='No' else '', subset=['Cumple L=2']))
    grupos_no_cumplen_l = df_l_diversity[df_l_diversity['Cumple L=2'] == 'No'].shape[0]
    if grupos_no_cumplen_l > 0:
        st.warning(f"⚠️ ¡Atención! Se detectaron {grupos_no_cumplen_l} grupos que no cumplen con l={l_actual}.")
    else:
        st.success(f"✅ Todos los grupos mostrados cumplen con l={l_actual}.")
    st.caption(f"La tabla muestra ejemplos de grupos y si cumplen la diversidad mínima (l={l_actual}) en el atributo sensible 'type'.")


# --- Contenido Pestaña 3: Privacidad Diferencial ---
with tab3:
    st.header("Detalles de Privacidad Diferencial (DP)")
    st.metric("Épsilon (ε) Utilizado", f"{epsilon_actual:.1f}", help="Valor más bajo indica mayor privacidad, pero puede afectar más la utilidad.")
    st.markdown(f"""
    Se aplicó Privacidad Diferencial durante el entrenamiento del modelo (simulado) utilizando la biblioteca Opacus (o similar)
    con un presupuesto de privacidad **ε = {epsilon_actual:.1f}**.

    **Parámetros Clave (Ejemplo Opacus):**
    * `noise_multiplier`: (Calculado en base a ε, delta, épocas, etc. - no mostrado aquí)
    * `max_grad_norm`: (Límite de la norma del gradiente - no mostrado aquí)
    * `delta (δ)`: Probabilidad de fallo de la privacidad (generalmente muy pequeña, ej. 1/N_samples - no mostrado aquí)

    **Interpretación de ε:**
    Un valor de ε={epsilon_actual:.1f} se considera generalmente un buen equilibrio entre privacidad y utilidad para muchas tareas,
    aunque la elección óptima depende del caso de uso específico y los requisitos regulatorios.
    """)

    # Slider interactivo (conceptual, no afecta a otros gráficos en esta simulación)
    epsilon_simulado = st.slider("Simular cambio de Épsilon (ε)", min_value=0.1, max_value=10.0, value=epsilon_actual, step=0.1,
                                 help="Mueve este deslizador para ver cómo cambiaría el nivel de privacidad (conceptual).")
    if epsilon_simulado < 1.0:
        st.info("Nivel de Privacidad: Muy Alto (Utilidad potencialmente muy afectada)")
    elif epsilon_simulado < 3.0:
        st.info("Nivel de Privacidad: Alto (Equilibrio común)")
    elif epsilon_simulado < 7.0:
        st.info("Nivel de Privacidad: Moderado")
    else:
        st.info("Nivel de Privacidad: Bajo (Utilidad potencialmente menos afectada)")


# --- Contenido Pestaña 4: Rendimiento Modelo ---
with tab4:
    st.header("Evaluación del Rendimiento del Modelo Anonimizado")
    st.subheader(f"Métricas Principales (Modelo con ε={epsilon_actual:.1f})")

    # Mostrar métricas clave
    col_m1, col_m2, col_m3 = st.columns(3)
    # Calcular sensibilidad (Recall) y especificidad
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1_score = 2 * (precision_anon/100 * sensitivity) / (precision_anon/100 + sensitivity) if (precision_anon/100 + sensitivity) > 0 else 0

    col_m1.metric("Precisión Global", f"{precision_anon:.1f}%")
    col_m2.metric("Sensibilidad (Recall)", f"{sensitivity:.1%}", help="Capacidad de detectar fraudes reales (TP / (TP + FN)).")
    col_m3.metric("F1-Score", f"{f1_score:.3f}", help="Media armónica de precisión y sensibilidad.")

    st.divider()

    col_g1, col_g2 = st.columns(2)

    with col_g1:
        st.subheader("Matriz de Confusión")
        fig_cm, ax_cm = plt.subplots(figsize=(5, 4))
        cax = ax_cm.matshow(cm, cmap=plt.cm.Blues)
        fig_cm.colorbar(cax)
        # Poner etiquetas
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax_cm.text(j, i, str(cm[i, j]), va='center', ha='center', color='black' if cm[i, j] < cm.max()/2 else 'white') # Ajustar color texto
        ax_cm.set_xlabel('Predicción')
        ax_cm.set_ylabel('Valor Real')
        ax_cm.set_xticks([0, 1])
        ax_cm.set_yticks([0, 1])
        ax_cm.set_xticklabels(['No Fraude', 'Fraude'])
        ax_cm.set_yticklabels(['No Fraude', 'Fraude'])
        ax_cm.xaxis.set_label_position('top')
        ax_cm.xaxis.tick_top()
        st.pyplot(fig_cm)

    with col_g2:
        st.subheader("Curva ROC")
        fig_roc, ax_roc = plt.subplots(figsize=(5, 4))
        ax_roc.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
        ax_roc.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Azar')
        ax_roc.set_xlim([0.0, 1.0])
        ax_roc.set_ylim([0.0, 1.05])
        ax_roc.set_xlabel('Tasa de Falsos Positivos (FPR)')
        ax_roc.set_ylabel('Tasa de Verdaderos Positivos (TPR)')
        ax_roc.set_title('Receiver Operating Characteristic')
        ax_roc.legend(loc="lower right")
        st.pyplot(fig_roc)

# --- Contenido Pestaña 5: Auditoría ---
with tab5:
    st.header("Auditoría y Logs")
    st.markdown("Registro de acciones relevantes realizadas sobre el modelo y los parámetros de privacidad.")
    st.dataframe(df_logs, use_container_width=True) # Usar todo el ancho

# --- Barra Lateral ---
st.sidebar.image("https://placehold.co/150x80/5cb85c/FFFFFF?text=GDPR", caption="Logo Placeholder") # Placeholder para logo
st.sidebar.info("Dashboard Conceptual - TFM Anonimización LLM/GDPR")
st.sidebar.markdown("---")
st.sidebar.header("Parámetros Actuales")
st.sidebar.markdown(f"**K-Anonimato:** {k_actual}")
st.sidebar.markdown(f"**L-Diversidad:** {l_actual}")
st.sidebar.markdown(f"**Épsilon (DP):** {epsilon_actual:.1f}")

ModuleNotFoundError: No module named 'matplotlib'